**INFORMATION SECURITY & PRIVACY**  

**HOMEWORK #2 (WEEK 4)**

# Importation du dataset

In [126]:
import pandas as pd 
import numpy as np

data = pd.read_csv('https://raw.githubusercontent.com/khandid3/isp.data/master/records.csv')
data.head(500)

,id,birth_year,sex,zipcode,region,canton,condition
0,0,2004,M,7168,Region Surselva,GR,Heart disease
1,5863,1972,M,7168,Region Surselva,GR,Heart disease
2,6216,1979,M,7168,Region Surselva,GR,Heart disease
3,18330,1980,M,7168,Region Surselva,GR,Heart disease
4,44387,1979,M,7168,Region Surselva,GR,Heart disease
...,...,...,...,...,...,...,...
495,98959,2009,F,3273,Verwaltungskreis Seeland,BE,Heart disease
496,27245,1970,M,1345,District du Jura-Nord vaudois,VD,Heart disease
497,28332,1992,M,1345,District du Jura-Nord vaudois,VD,Heart disease
498,38610,2007,M,1345,District du Jura-Nord vaudois,VD,Heart disease


In [127]:
data.dtypes

id             int64
birth_year     int64
sex           object
zipcode        int64
region        object
canton        object
condition     object
dtype: object

#Question 2

In [128]:
data[(data["birth_year"] == 1970) & (data['sex'] == 'F') & (data['zipcode'] == 1092)]

,id,birth_year,sex,zipcode,region,canton,condition
12526,8905,1970,F,1092,District de Lavaux-Oron,VD,Heart disease
72223,35284,1970,F,1092,District de Lavaux-Oron,VD,Cancer


In [129]:
data[(data["birth_year"] == 1978) & (data['sex'] == 'M') & (data['zipcode'] == 1092)]

,id,birth_year,sex,zipcode,region,canton,condition
50217,61047,1978,M,1092,District de Lavaux-Oron,VD,Flu


#Question 3

In [130]:
k_anonymity = data.copy()
k_anonymity['size'] = ""
k_anonymity = k_anonymity.groupby(['sex', 'zipcode'], as_index =False)['size'].count()
k_anonymity

,sex,zipcode,size
0,F,1000,12
1,F,1003,23
2,F,1004,17
3,F,1005,15
4,F,1006,16
...,...,...,...
6359,M,9652,18
6360,M,9655,19
6361,M,9656,19
6362,M,9657,16


# Question 4 & 5

In [131]:
k_anonymity

,sex,zipcode,size
0,F,1000,12
1,F,1003,23
2,F,1004,17
3,F,1005,15
4,F,1006,16
...,...,...,...
6359,M,9652,18
6360,M,9655,19
6361,M,9656,19
6362,M,9657,16


In [132]:
min(k_anonymity['size'])

5

k = 5

# Question 6

Nous devons à présent générer les 12 possibilites de généralisation.
On peut les trouver manuellement, ce sont:


- (0,0,0)
- (0,0,1)
- (0,0,2)
- (0,1,0)
- (0,1,1)
- (0,1,2)
- (1,0,0)
- (1,0,1)
- (1,0,2)
- (1,1,0)
- (1,1,1)
- (1,1,2)

Ensuite, nous appliquerons chacune de ces généralisations à notre dataset.

Puis, nous garderons les généralisations où k >= 6.

Enfin, avec les triplets qui nous restent, nous sélectionnerons celui avec la distortion la plus basse. 



# Question 7

## Marche à suivre

Nous allons procéder comme suit:     

- Appliquer à notre dataframe les 12 généralisations possibles ;
- Garder les généralisations avec k >= 6 ;
- Calculer la distortion sur chacune de ces généralisations ;
- Garder la généralistion avec k >= 6 & qui a la distortion la plus basse.



## Préparation du dataframe

In [133]:
# Faisons une copie du dataframe 'data' => 'new_data'.
new_data = data.copy()
# Gardons les colonnes nécessaires et éliminons les autres
new_data.drop(['region', 'canton', 'condition', 'id'], axis=1, inplace=True)
# Créons une colonne 'size' vide qui sera complétée par la suite par la cardinalité des classes d'équivalence
new_data['size'] = ''
# On convertit les colonnes 'birth_year', et 'zipcode' en string afin de les modifier plus aisément par la suite (cf. notre fonction lambda dans les généralisations)
new_data['birth_year'] = new_data['birth_year'].astype('str')
new_data['zipcode'] = new_data['zipcode'].astype('str')
new_data.dtypes

birth_year    object
sex           object
zipcode       object
size          object
dtype: object

In [171]:
new_data

,birth_year,sex,zipcode,size
0,2004,M,7168,
1,1972,M,7168,
2,1979,M,7168,
3,1980,M,7168,
4,1979,M,7168,
...,...,...,...,...
99995,1970,M,3805,
99996,2015,F,3805,
99997,1994,F,3929,
99998,1987,F,3929,


##Généralisation 1 : (1, 0, 0)

In [135]:
# On crée une copie du dataframe 'new_data' afin de travailler dessus sans modifier le dataframe de base et le copier de nouveau par la suite pour les autres généralisations
new_data1 = new_data.copy()
# On généralise de niveau 1 birth_year
# On supprime le dernier digit de birth_year
new_data1['birth_year'] = new_data1['birth_year'].apply(lambda x: x[0:3])
# On affiche new_data1
new_data1

,birth_year,sex,zipcode,size
0,200,M,7168,
1,197,M,7168,
2,197,M,7168,
3,198,M,7168,
4,197,M,7168,
...,...,...,...,...
99995,197,M,3805,
99996,201,F,3805,
99997,199,F,3929,
99998,198,F,3929,


In [136]:
# on regroupe les données en fonction des QID et on calcule la cardinalité ('size') de chacune des classes d'équivalence ainsi créées.
resultat1 = new_data1.groupby(['birth_year', 'sex', 'zipcode'], as_index =False)['size'].count()
resultat1

,birth_year,sex,zipcode,size
0,197,F,1000,2
1,197,F,1003,3
2,197,F,1004,3
3,197,F,1005,4
4,197,F,1006,3
...,...,...,...,...
32893,202,M,9651,1
32894,202,M,9652,2
32895,202,M,9655,2
32896,202,M,9656,1


In [137]:
# Vérifions qu'on ait bien 3 résultats pour 'birth_year' == 197 et zipcode == 1006
new_data1[(new_data1['birth_year'] == '197') & (new_data3['zipcode'] == '1006')]

,birth_year,sex,zipcode,size
63395,197,F,1006,
63396,197,F,1006,
77438,197,F,1006,


C'est bel et bien le cas ! Ufff

In [138]:
# On trouve la taille la plus petite, i.e. notre k pour cette généralisation
resultat1['size'].min()

1

k = 1

Dorénavant, pour les autres généralisations, il y aura moins de commentaires car la logique applicative reste la même, mais pour certaines il y a plus de généralisation à faire (e.g. 'sex', 'zipcode')

## Généralisation 2 : (1, 1, 1)


In [139]:
new_data2 = new_data.copy()
# generalizing birth_year level 1
new_data2['birth_year'] = new_data2['birth_year'].apply(lambda x: x[0:3])
# generalizing sex level 1
new_data2['sex'] = "*"
#generalizing zipcode level 1
new_data2["zipcode"] = data["region"]
new_data2

,birth_year,sex,zipcode,size
0,200,*,Region Surselva,
1,197,*,Region Surselva,
2,197,*,Region Surselva,
3,198,*,Region Surselva,
4,197,*,Region Surselva,
...,...,...,...,...
99995,197,*,Verwaltungskreis Interlaken-Oberhasli,
99996,201,*,Verwaltungskreis Interlaken-Oberhasli,
99997,199,*,Bezirk Visp,
99998,198,*,Bezirk Visp,


In [140]:
resultat2 = new_data2.groupby(['birth_year', 'sex', 'zipcode'], as_index =False)['size'].count()
resultat2

,birth_year,sex,zipcode,size
0,197,*,Arrondissement administratif Jura bernois,292
1,197,*,Bezirk Aarau,106
2,197,*,Bezirk Affoltern,95
3,197,*,Bezirk Andelfingen,199
4,197,*,Bezirk Arbon,94
...,...,...,...,...
853,202,*,Wahlkreis Sursee,36
854,202,*,Wahlkreis Toggenburg,36
855,202,*,Wahlkreis Werdenberg,18
856,202,*,Wahlkreis Wil,15


In [141]:
resultat2['size'].min()

1

k = 1

##Généralisation 3 : (1, 1, 0) 

In [142]:
new_data3 = new_data.copy()
# generalizing birth_year level 1
new_data3['birth_year'] = new_data3['birth_year'].apply(lambda x: x[0:3])
# generalizing sex level 1
new_data3['sex'] = "*"
new_data3

,birth_year,sex,zipcode,size
0,200,*,7168,
1,197,*,7168,
2,197,*,7168,
3,198,*,7168,
4,197,*,7168,
...,...,...,...,...
99995,197,*,3805,
99996,201,*,3805,
99997,199,*,3929,
99998,198,*,3929,


In [143]:
resultat3 = new_data3.groupby(['birth_year', 'sex', 'zipcode'], as_index =False)['size'].count()
resultat3

,birth_year,sex,zipcode,size
0,197,*,1000,3
1,197,*,1003,6
2,197,*,1004,8
3,197,*,1005,8
4,197,*,1006,3
...,...,...,...,...
17928,202,*,9651,1
17929,202,*,9652,2
17930,202,*,9655,3
17931,202,*,9656,1


In [144]:
resultat3['size'].min()

1

k = 1

##Généralisation 4 : (1, 0, 1) 

In [145]:
new_data4 = new_data.copy()
# generalizing birth_year level 1
new_data4['birth_year'] = new_data4['birth_year'].apply(lambda x: x[0:3])
#generalizing zipcode level 1
new_data4["zipcode"] = data["region"]
new_data4

,birth_year,sex,zipcode,size
0,200,M,Region Surselva,
1,197,M,Region Surselva,
2,197,M,Region Surselva,
3,198,M,Region Surselva,
4,197,M,Region Surselva,
...,...,...,...,...
99995,197,M,Verwaltungskreis Interlaken-Oberhasli,
99996,201,F,Verwaltungskreis Interlaken-Oberhasli,
99997,199,F,Bezirk Visp,
99998,198,F,Bezirk Visp,


In [146]:
resultat4 = new_data4.groupby(['birth_year', 'sex', 'zipcode'], as_index =False)['size'].count()
resultat4

,birth_year,sex,zipcode,size
0,197,F,Arrondissement administratif Jura bernois,157
1,197,F,Bezirk Aarau,49
2,197,F,Bezirk Affoltern,59
3,197,F,Bezirk Andelfingen,106
4,197,F,Bezirk Arbon,49
...,...,...,...,...
1707,202,M,Wahlkreis Sursee,20
1708,202,M,Wahlkreis Toggenburg,19
1709,202,M,Wahlkreis Werdenberg,7
1710,202,M,Wahlkreis Wil,4


In [147]:
resultat4['size'].min()

1

k = 1

##Généralisation 5 : (0, 1, 0) 

In [148]:
new_data5 = new_data.copy()
# generalizing sex level 1
new_data5['sex'] = "*"
new_data5


,birth_year,sex,zipcode,size
0,2004,*,7168,
1,1972,*,7168,
2,1979,*,7168,
3,1980,*,7168,
4,1979,*,7168,
...,...,...,...,...
99995,1970,*,3805,
99996,2015,*,3805,
99997,1994,*,3929,
99998,1987,*,3929,


In [149]:
resultat5 = new_data5.groupby(['birth_year', 'sex', 'zipcode'], as_index =False)['size'].count()
resultat5

,birth_year,sex,zipcode,size
0,1970,*,1003,1
1,1970,*,1004,1
2,1970,*,1010,2
3,1970,*,1023,1
4,1970,*,1030,1
...,...,...,...,...
74923,2021,*,9612,1
74924,2021,*,9614,1
74925,2021,*,9630,1
74926,2021,*,9652,1


In [150]:
resultat5['size'].min()

1

k = 1

##Généralisation 6 : (0, 1, 1) 

In [151]:
new_data6 = new_data.copy()
# generalizing sex level 1
new_data6['sex'] = "*"
#generalizing zipcode level 1
new_data6["zipcode"] = data["region"]
new_data6

,birth_year,sex,zipcode,size
0,2004,*,Region Surselva,
1,1972,*,Region Surselva,
2,1979,*,Region Surselva,
3,1980,*,Region Surselva,
4,1979,*,Region Surselva,
...,...,...,...,...
99995,1970,*,Verwaltungskreis Interlaken-Oberhasli,
99996,2015,*,Verwaltungskreis Interlaken-Oberhasli,
99997,1994,*,Bezirk Visp,
99998,1987,*,Bezirk Visp,


In [152]:
resultat6 = new_data6.groupby(['birth_year', 'sex', 'zipcode'], as_index =False)['size'].count()
resultat6

,birth_year,sex,zipcode,size
0,1970,*,Arrondissement administratif Jura bernois,25
1,1970,*,Bezirk Aarau,15
2,1970,*,Bezirk Affoltern,10
3,1970,*,Bezirk Andelfingen,18
4,1970,*,Bezirk Arbon,16
...,...,...,...,...
7343,2021,*,Wahlkreis Sursee,15
7344,2021,*,Wahlkreis Toggenburg,12
7345,2021,*,Wahlkreis Werdenberg,5
7346,2021,*,Wahlkreis Wil,6


In [153]:
resultat6['size'].min()

1

k = 1

##Généralisation 7 : (0, 1, 2) 

In [172]:
new_data7 = new_data.copy()
# generalizing sex level 1
new_data7['sex'] = "*"
#generalizing zipcode level 2
new_data7["zipcode"] = data["canton"]
new_data7

,birth_year,sex,zipcode,size
0,2004,*,GR,
1,1972,*,GR,
2,1979,*,GR,
3,1980,*,GR,
4,1979,*,GR,
...,...,...,...,...
99995,1970,*,BE,
99996,2015,*,BE,
99997,1994,*,VS,
99998,1987,*,VS,


In [173]:
resultat7 = new_data7.groupby(['birth_year', 'sex', 'zipcode'], as_index =False)['size'].count()
resultat7

,birth_year,sex,zipcode,size
0,1970,*,AG,138
1,1970,*,AI,5
2,1970,*,AR,13
3,1970,*,BE,264
4,1970,*,BL,61
...,...,...,...,...
1345,2021,*,UR,8
1346,2021,*,VD,149
1347,2021,*,VS,86
1348,2021,*,ZG,8


In [174]:
resultat7['size'].min()

1

k = 1

## Généralisation 8 : (1, 1, 2)

In [175]:
new_data8 = new_data.copy()
# generalizing 'birth_year' level 1
new_data8['birth_year'] = new_data8['birth_year'].apply(lambda x: x[0:3])
#generalizing 'sex' level 1
new_data8['sex'] = "*"
# generalizing 'zipcode' level 2
new_data8["zipcode"] = data["canton"]
new_data8

,birth_year,sex,zipcode,size
0,200,*,GR,
1,197,*,GR,
2,197,*,GR,
3,198,*,GR,
4,197,*,GR,
...,...,...,...,...
99995,197,*,BE,
99996,201,*,BE,
99997,199,*,VS,
99998,198,*,VS,


In [176]:
resultat8 = new_data8.groupby(['birth_year', 'sex', 'zipcode'], as_index =False)['size'].count()
resultat8

,birth_year,sex,zipcode,size
0,197,*,AG,1438
1,197,*,AI,40
2,197,*,AR,137
3,197,*,BE,2814
4,197,*,BL,534
...,...,...,...,...
151,202,*,UR,27
152,202,*,VD,333
153,202,*,VS,205
154,202,*,ZG,17


In [177]:
resultat8["size"].min()

6

k = 6

##Généralisation 9 : (0, 0 ,1) 

In [178]:
new_data9 = new_data.copy()
# generalizing 'zipcode' level 1
new_data9["zipcode"] = data["region"]
new_data9

,birth_year,sex,zipcode,size
0,2004,M,Region Surselva,
1,1972,M,Region Surselva,
2,1979,M,Region Surselva,
3,1980,M,Region Surselva,
4,1979,M,Region Surselva,
...,...,...,...,...
99995,1970,M,Verwaltungskreis Interlaken-Oberhasli,
99996,2015,F,Verwaltungskreis Interlaken-Oberhasli,
99997,1994,F,Bezirk Visp,
99998,1987,F,Bezirk Visp,


In [179]:
resultat9 = new_data9.groupby(['birth_year', 'sex', 'zipcode'], as_index =False)['size'].count()
resultat9

,birth_year,sex,zipcode,size
0,1970,F,Arrondissement administratif Jura bernois,13
1,1970,F,Bezirk Aarau,6
2,1970,F,Bezirk Affoltern,7
3,1970,F,Bezirk Andelfingen,11
4,1970,F,Bezirk Arbon,10
...,...,...,...,...
14288,2021,M,Wahlkreis Sursee,8
14289,2021,M,Wahlkreis Toggenburg,7
14290,2021,M,Wahlkreis Werdenberg,2
14291,2021,M,Wahlkreis Wil,2


In [180]:
resultat9['size'].min()

1

k = 1

##Généralisation 10 : (0, 0 ,2) 

In [181]:
new_data10 = new_data.copy()
# generalizing 'zipcode' level 2
new_data10["zipcode"] = data["canton"]
new_data10

,birth_year,sex,zipcode,size
0,2004,M,GR,
1,1972,M,GR,
2,1979,M,GR,
3,1980,M,GR,
4,1979,M,GR,
...,...,...,...,...
99995,1970,M,BE,
99996,2015,F,BE,
99997,1994,F,VS,
99998,1987,F,VS,


In [182]:
resultat10 = new_data10.groupby(['birth_year', 'sex', 'zipcode'], as_index =False)['size'].count()
resultat10

,birth_year,sex,zipcode,size
0,1970,F,AG,71
1,1970,F,AI,2
2,1970,F,AR,6
3,1970,F,BE,136
4,1970,F,BL,34
...,...,...,...,...
2678,2021,M,UR,6
2679,2021,M,VD,69
2680,2021,M,VS,41
2681,2021,M,ZG,3


In [183]:
resultat10['size'].min()

1

k = 1

##Généralisation 11 : (1, 0 ,2) 

In [184]:
new_data11 = new_data.copy()
# generalizing 'birth_year' level 1
new_data11['birth_year'] = new_data11['birth_year'].apply(lambda x: x[0:3])
# generalizing 'zipcode' level 2
new_data11["zipcode"] = data["canton"]
new_data11

,birth_year,sex,zipcode,size
0,200,M,GR,
1,197,M,GR,
2,197,M,GR,
3,198,M,GR,
4,197,M,GR,
...,...,...,...,...
99995,197,M,BE,
99996,201,F,BE,
99997,199,F,VS,
99998,198,F,VS,


In [185]:
resultat11 = new_data11.groupby(['birth_year', 'sex', 'zipcode'], as_index =False)['size'].count()
resultat11

,birth_year,sex,zipcode,size
0,197,F,AG,748
1,197,F,AI,25
2,197,F,AR,74
3,197,F,BE,1480
4,197,F,BL,285
...,...,...,...,...
307,202,M,UR,17
308,202,M,VD,166
309,202,M,VS,106
310,202,M,ZG,7


In [186]:
resultat11['size'].min()

2

k = 2

## Généralisation 12 (0,0,0)

In [187]:
new_data12 = new_data.copy()
resultat12 = new_data12.groupby(['birth_year', 'sex', 'zipcode'], as_index =False)['size'].count()
resultat12

,birth_year,sex,zipcode,size
0,1970,F,1004,1
1,1970,F,1010,1
2,1970,F,1034,2
3,1970,F,1040,1
4,1970,F,1052,1
...,...,...,...,...
86364,2021,M,9606,1
86365,2021,M,9614,1
86366,2021,M,9630,1
86367,2021,M,9652,1


In [188]:
resultat12['size'].min()

1

k = 1

##Calcul de la distortion 

* Généralisation (0, 0, 0) : 

1/3 (0/1 + 0/1 + 0/2) = 0

*   Généralisation (1, 0, 0) : 

1/3 (1/1 + 0/1 + 0/2) = 1/3

*   Généralisation (1, 1, 1) : 

1/3 (1/1 + 1/1 + 1/2) = 1/3

*   Généralisation (1, 1, 0) :

1/3 (1/1 + 1/1 + 0/2) = 2/3

*   Généralisation (1, 0, 1) :

1/3 (1/1 + 0/1 + 1/2) = 0,5

*   Généralisation (0, 1, 0) :

1/3 (0/1 + 1/1 + 0/2) = 1/3

*   Généralisation (0, 1, 1) :

1/3 (0/1 + 1/1 + 1/2) = 0.5

*   Généralisation (0, 1, 2) :

1/3 (0/1 + 1/1 + 2/2) = 2/3

*   Généralisation (1, 1, 2) :

1/3 (1/1 + 1/1 + 2/2) = 1

*   Généralisation (0, 0, 1) :

1/3 (0/1 + 0/1 + 1/2) = 1/6

*   Généralisation (0, 0, 2) : 

1/3 (0/1 + 0/1 + 2/2) = 1/3

*   Généralisation (1, 0, 2) : 

1/3 (1/1 + 0/1 + 2/2) = 2/3

## Réponse finale

On obtient donc que la seule généralisation avec k >= 6 est la généralisation (1, 1, 2), i.e. la plus grande généralisation possible. 
Sa distortion est de 1.